In [2]:
import pandas as pd
import os
from glob import glob
import seaborn as sea
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt


`# Instructions
1. Create a new folder for your run. 
2. Before you drive, Save your VE table in that folder as 've.table'
3. Log your run (in ASCII format) as "log.msl"
4. Use thse functions to do things: 
    

In [3]:
def get_file(filename):

    with open(filename, errors = 'ignore') as f:
        data = f.read()
        
    data = data.split('\n')[2:]
    data = '\n'.join(data)
    with open('temp.csv', 'w') as f:
        f.write(data)
        
    data = pd.read_csv('temp.csv', sep = '\t', low_memory=False)
    dtypes = data.iloc[0]
    data = data.iloc[1:]
    
    for column in data.columns:
        try:
            data[column] = pd.to_numeric(data[column])
        except ValueError as e:
            pass
    
    data['true_target']  = data['AFR 1 Target'] * (1 - (data["Fuel: Accel enrich"]/100 - 1))
    data['error'] = data['AFR']- data['true_target']
    
    return data


In [4]:
def get_table(filename):
    

    tree = ET.parse(filename + '.table')
    
    
    for thing in tree.getiterator():
        if ('zValues' in thing.tag) and ('tableData' not in thing.tag):
            raw_data = thing.text

        if 'xAxis'in thing.tag:
            x_axis = thing.text
            x_axis = x_axis.strip().replace(' ', '').split('\n')
        if 'yAxis' in thing.tag:
            y_axis = thing.text
            y_axis = y_axis.strip().replace(' ', '').split('\n')


    rows = raw_data.strip().split('\n')
    raw_data_table = []

    for row in rows:
        vals = row.strip().split(' ')

        raw_data_table.append([float(x) for x in vals])

    data = pd.DataFrame(columns= [float(x) for x in x_axis], index= [float(y) for y in y_axis], data = raw_data_table )
    
    return data



In [5]:
def get_success_rate(folder):
    data = get_file(folder + '/log.msl')
    ve = get_table(folder+ '/ve')
    
    states = []
    for column in data.columns:
        if 'engine' in column.lower() and column != 'Engine':
            states.append(column)
    
    text= ['Positive is Lean, Negative is Rich']
    for state in states:
        temp = data[data[state] == 1]
        avg_error = round(temp['error'].mean(),2)
        samples = len(temp)
        text.append(f"Average Error: {avg_error} | samples: {samples} | {state}")
    
    avg_error = round(data['error'].mean(),2)
    text.append(f"AVERAGE ERROR {avg_error}")
    text = '\n'.join(text)

    with open(folder+ '/stats.txt', 'w') as f:
        f.write(text)
        
    cols_to_chart = []
    for col in data.columns:
        if 'fuel' in col.lower():
            if 'cor' in col:
                cols_to_chart.append(col)
                
    cols_to_chart.append("CLT")
    for col in cols_to_chart:
        if len(data[col].value_counts())> 1:
            fig, ax = plt.subplots()
            fig.set_size_inches(8*1.3,8)
            data.set_index('Time')[col].plot()
            ax.set_title(col)
            plt.savefig(folder + '/'+col + '.png', facecolor='w')
            
            plt.close()
            
    
    return text



In [6]:
def get_factor(data, ve, function): # FOR ALL CELLS IN VE TABLE, DO A FUNCTION TO THE DATA SUPPLIED
    final = pd.DataFrame(index = ve.index, columns= ve.columns)
    start_rpm = 0
    
    for end_rpm in ve.columns: # Iterate through each RPM group
        start_load = 0
        
        for end_load in ve.index: # Iterate through each Load Group
            
            sample = data[(data['Load'] >= start_load) & (data['Load'] < end_load)]#Filter for Load
            if len(sample) > 0:
                sample = sample[(sample['RPM'] >= start_rpm) & (sample['RPM'] < end_rpm)] #Filter for RPM
                if len(sample) > 0:
                    result = function(sample)# Calculate on remaining data
                    final.loc[end_load, end_rpm] = result #Save result on table
            start_load = end_load #Move Load iterator forward
        start_rpm = end_rpm#Move RPM iterator forward once every load iterator has been hit for that RPM.
    return final

In [7]:

def calculate_predicted_ve_table(samples, errors,ve, avg_tgt_afr,  min_samples):
    # Remove Minimum Samples
    sample_mask = samples.copy()
    sample_mask[samples<= min_samples] = 0 
    sample_mask[samples > min_samples] = 1
    errors = errors * sample_mask


    # Calculate PCT that everything is off
    # errors are errors/avg_tgt to convert to pct. 
    # One is added so that we can multiply the ve table by it and scale it
    # fillna is so that unknown values are kept the same as original ve table.
    pred_ve = (1 + (errors/avg_tgt_afr) ).fillna(1) * ve
    
    # Clean up for paste to clipboard
    pred_ve = pred_ve.sort_index(ascending = False).apply(lambda x: round(x,1))
    print("RECCOMENDED VE TABLE ADDED TO CLIPBOARD")
    pred_ve.to_clipboard(index = False, header = False)
    
    # Return to regular ordering
    pred_ve = pred_ve.sort_index()
    return pred_ve

In [13]:
data = get_file(folder + '/log.msl')

In [23]:
(data["Load"] - data['MAP']).mean()

-0.0011588604538870198

In [8]:
def get_suggestions_based_on(folder):
    data = get_file(folder + '/log.msl')
    ve = get_table(folder+ '/ve')
    
    # SPECIFIC FILTERS HERE - Use Drive Notes to determine________________________________________
    data = data[data['CLT'] > 145] # Ensure car is warmed up - General 
    #data = data[(data["RPM"] > 2000) | (data['Load'] > 45)] # Avoid messing with IDLE - General
    #data = data[data['Engine in overrun'] != 1]  # Avoid  Overrun Tuning - General
    data['error'] = data['error'] 
    ################################################################______________________________
    
    # Calculate mean error:
    def get_avg_error(x):
        return x['error'].mean() # Reduce IMpact of errors
    errors = get_factor(data, ve, get_avg_error)
    
    #Scale errors by acceleration enrichment - 
    
    #Calculate count of samples
    samples = get_factor(data, ve, len)
    
    avg_tgt_afr = get_factor(data, ve, lambda x: x['AFR 1 Target'].mean()) # Errors are a pct of this.
    pred_ve = calculate_predicted_ve_table(samples,
                                           errors,
                                           ve,
                                           avg_tgt_afr,
                                           min_samples=3)
    
    
    ################ 
    return pred_ve

    

In [9]:
folder = 'oct_3'

In [10]:
print(get_success_rate(folder))

Positive is Lean, Negative is Rich
Average Error: 1.16 | samples: 1034 | Engine in cruise state
Average Error: 0.23 | samples: 494 | Engine accelerating slowly
Average Error: 0.38 | samples: 683 | Engine decelerating slowly
Average Error: 0.96 | samples: 657 | Engine in overrun
Average Error: -0.29 | samples: 1473 | Engine idling
Average Error: 0.35 | samples: 362 | Engine WOT
AVERAGE ERROR 0.4


In [11]:
pred_ve = get_suggestions_based_on(folder)
ve = get_table(folder + '/ve')


change_map = (pred_ve - ve )/ve # As a percent of current position
change_map = (change_map + 1) * 100 # Proper PCT
change_map = change_map.sort_index(ascending= False)
change_map = change_map.apply(lambda x: round(x, 2))

RECCOMENDED VE TABLE ADDED TO CLIPBOARD


In [12]:

fig, ax = plt.subplots()
ax.set_title("Blue means adding more fuel, red means cutting fuel")
fig.set_size_inches(8*1.3, 8)
sea.heatmap(change_map, center = 100, cmap='RdBu')
ax.set_xlabel('RPM')
ax.set_ylabel("Load");
plt.savefig(folder + '/change_pic.png', facecolor='w')
            